# ID3 算法

## 算法步骤

对于一个数据集,生成节点的算法如下：
* 1.如果没有属性就采用多数表决
* 2.如果属于同一个类，则算法停止，使用该标记做分类 
* 3.根据最大信息增益的属性划分子数据集，对每个数据集生成子节点

代码实现如下：

In [1]:
import pandas as pd
import numpy as np

In [28]:
def gTree(df):
    print(df)
    if(ispure(df)): #无属性
        return gPureNode(df)
    
    if(issame(df)): #同一类
        return gSameNode(df)
        
    maxindex=getMaxIndex(df)
    # 划分
    subdfs=df.groupby(maxindex)
    
    #生成节点和子节点
    node={}
    node["分组属性"]=maxindex
    node["子节点"]=[]
    for subdf in subdfs:
        sdf=subdf[1]
        sdf.pop(maxindex)
        node["子节点"].append(gTree(sdf))
        
    return node

可以看到代码的逻辑基本和算法描述一样，接下来一个一个实现功能函数就行

判断是不是空属性和多数表决比较简单：

In [29]:
def ispure(df):
    return df.shape[1]==1
print(ispure(pd.DataFrame(np.zeros((3,10)))))
print(ispure(pd.DataFrame(np.zeros((3,1)))))

False
True


In [30]:
def maxType(df):
    """
    多数表决
    实现得不够优雅
    """
    maxcount=-1
    maxtype=None
    for sdf in df.groupby(df.columns[0]):
        if(sdf[1].count()[0] >maxcount):
            maxcount=sdf[1].count()[0]
            maxtype=sdf[0]
    return maxtype
(maxType(pd.DataFrame(np.array([1,1,1,1,1,2,2,2,3]).reshape((9,1))))
,maxType(pd.DataFrame(np.array([1,1,1,1,2,2,2,2,3]).reshape((9,1))))
,maxType(pd.DataFrame(np.array([1,1,1,2,2,2,2,2,3]).reshape((9,1)))))

(1, 1, 2)

In [31]:
def issame(df):
    return df[df.columns[-1]].unique().size==1
(issame(pd.DataFrame(np.array([4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4]).reshape(9,2))),
 issame(pd.DataFrame(np.array([4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4]).reshape(9,2))))

(True, False)

In [41]:
def gPureNode(df):
    types=maxType(df)
    node={}
    node["type"]=types
    return node
def gSameNode(df):
    types=df[df.columns[-1]].values[0]
    node={}
    node["type"]=types
    return node;

In [33]:
def getMaxIndex(df):
    if (df.columns.size<=2):
        return df.columns[0]
    
    allindex=df.columns[:-1]
    maxindex=allindex[0]
    maxp=getP(df,maxindex)
    for i in allindex[1:0]:
        p=getP(df,maxindex)
        if(p<maxp):
            maxp=p
            maxindex=i
    return maxindex

In [34]:
names = ['Bob','Jessica','Mary','John','Mel']
sex = ['Bob','Jessica','Mary','John','Mel']
births = [968, 155, 77, 578, 973]
BabyDataSet = list(zip(names,births,sex))
df = pd.DataFrame(data = BabyDataSet, columns=['Names', "sex",'Births'])

In [35]:
df.columns.size

3

In [36]:
from math import log
def getP(df,index):
    
    classname=df.columns[-1]
    theP=0.0
    allcount=df.shape[0]
    for v in df[index].unique():
        subdf=df[df[index]==v]
        subcount=subdf.shape[0]
        subP=0.0
        for subclass in subdf[classname].unique():
            subclasscount=subdf[subdf[classname]==subclass].shape[0]
            p=subclasscount / subcount
            subP+= -(p) * log(p,2)
            #print(v,subclass,p)
       
        theP+=(subcount / allcount)*subP
    return theP

In [43]:
outlook=[1,1,2,3,3,3,2,1,1,3,1,2,2,3]
temp=[1,1,1,2,3,3,3,2,3,2,2,2,1,2]
hum=[1,1,1,1,2,2,2,1,2,2,2,1,2,1]
windy=[0,1,0,0,0,1,1,0,0,0,1,1,0,0]
play=[0,0,1,1,1,0,1,0,1,1,1,1,1,0]
DataSet = list(zip(outlook,temp,hum,windy,play))
playdf = pd.DataFrame(data = DataSet, columns=["outlook","temp","hum","windy","play"])

In [44]:
playdf

,outlook,temp,hum,windy,play
0,1,1,1,0,0
1,1,1,1,1,0
2,2,1,1,0,1
3,3,2,1,0,1
4,3,3,2,0,1
5,3,3,2,1,0
6,2,3,2,1,1
7,1,2,1,0,0
8,1,3,2,0,1
9,3,2,2,0,1


In [45]:
getP(playdf,"outlook")

0.6935361388961918

In [46]:
gTree(playdf)

    outlook  temp  hum  windy  play
0         1     1    1      0     0
1         1     1    1      1     0
2         2     1    1      0     1
3         3     2    1      0     1
4         3     3    2      0     1
5         3     3    2      1     0
6         2     3    2      1     1
7         1     2    1      0     0
8         1     3    2      0     1
9         3     2    2      0     1
10        1     2    2      1     1
11        2     2    1      1     1
12        2     1    2      0     1
13        3     2    1      0     0
    temp  hum  windy  play
0      1    1      0     0
1      1    1      1     0
7      2    1      0     0
8      3    2      0     1
10     2    2      1     1
   hum  windy  play
0    1      0     0
1    1      1     0
    hum  windy  play
7     1      0     0
10    2      1     1
   windy  play
7      0     0
    windy  play
10      1     1
   hum  windy  play
8    2      0     1
    temp  hum  windy  play
2      1    1      0     1
6      3    2      

{'分组属性': 'outlook',
 '子节点': [{'分组属性': 'temp',
   '子节点': [{'type': 0},
    {'分组属性': 'hum', '子节点': [{'type': 0}, {'type': 1}]},
    {'type': 1}]},
  {'type': 1},
  {'分组属性': 'temp',
   '子节点': [{'分组属性': 'hum',
     '子节点': [{'分组属性': 'windy', '子节点': [{'type': 0}]}, {'type': 1}]},
    {'分组属性': 'hum',
     '子节点': [{'分组属性': 'windy', '子节点': [{'type': 1}, {'type': 0}]}]}]}]}